<a href="https://colab.research.google.com/github/AbdullahMakhdoom/Image-Search-Engine/blob/main/improve_search_time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Objective** : In this notebook, we will benchmark various Nearest Neighbors algorithms based on the time it takes to index and locate the most similar image based on the features of Caltech101.
We will also see the improvement in search time when features are reduced by PCA. Aditionally, we will use a C++ library with python binding called Annoy and observe it's effect in speeding up the query time.

In [2]:
import numpy as np
import pickle
from tqdm import tqdm, tqdm_notebook
import random
import time
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import PIL
from PIL import Image
from sklearn.neighbors import NearestNeighbors

import glob
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

## Standard Features

Experimentation will be done on standard ResNet-50 features from the images of Caltech101 dataset. (extracted in `feature_extraction.ipynb`)

In [3]:
filenames = pickle.load(open('/content/drive/MyDrive/Caltech101-features/filenames-caltech101.pickle', 'rb'))
feature_list = pickle.load(open('/content/drive/MyDrive/Caltech101-features/features-caltech101-resnet.pickle',
                                'rb'))
class_ids = pickle.load(open('/content/drive/MyDrive/Caltech101-features/class_ids-caltech101.pickle', 'rb'))

In [4]:
num_images = len(filenames)
num_features_per_image = len(feature_list[0])
print("Number of images = ", num_images)
print("Number of features per image = ", num_features_per_image)

Number of images =  8677
Number of features per image =  2048


In [5]:
random_image_index = random.randint(0, num_images)

### Standard Features + Brute Force on one image

In [ ]:
%timeit NearestNeighbors(n_neighbors = 5, algorithm='brute', metric='euclidean')


The slowest run took 6.22 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 3.44 µs per loop


In [ ]:
# timit command does not store variable in memory
# so we need to re-run the same command and store the results
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)
                             

In [ ]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

10 loops, best of 5: 82.6 ms per loop


### Standard Features + k-d Tree Algorithm on one image

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list)


1 loop, best of 5: 2.74 s per loop


In [ ]:
neighbors = NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list)

In [ ]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

10 loops, best of 5: 37.4 ms per loop


### Standard Features + Ball Tree Algorithm on one image

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list)


1 loop, best of 5: 2.17 s per loop


In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)

In [ ]:
%timeit neighbors.kneighbors([feature_list[random_image_index]])

10 loops, best of 5: 25.4 ms per loop


Let's experiment on a random set of 1000 images.

In [ ]:
random_image_indices = random.sample(range(0, num_images), 1000)
random_feature_list = [
    feature_list[each_index] for each_index in random_image_indices]

### Standard features + Brute Force on 1000 images


In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)                             

1 loop, best of 5: 1.47 s per loop


### Standard features +  k-d Tree  on 1000 images

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

1 loop, best of 5: 36.7 s per loop


### Standard features +  Ball Tree on 1000 images

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='ball_tree').fit(feature_list)
%timeit neighbors.kneighbors(random_feature_list)

1 loop, best of 5: 24.7 s per loop


## Dimensionality Reduction using PCA
 Using Principle Component Analysis(PCA), we reduce the feature size to 100, and thus further speed up search query time.
 

In [14]:
num_feature_dimensions = 100

In [15]:
pca = PCA(n_components=num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
feature_list_compressed = feature_list_compressed.tolist()


In [ ]:
print(pca.explained_variance_ratio_[0:20])


[0.06110197 0.04382468 0.04060568 0.03228543 0.02124298 0.01967341
 0.01750924 0.01519272 0.01506694 0.01313028 0.01261717 0.01226299
 0.01129626 0.01055883 0.00959002 0.0093974  0.00869048 0.00849483
 0.008367   0.00772746]


The numbers displayed above show the relative importance of the first 20 features.

### PCA + Brute Force Algorithm on one image

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='brute', metric='euclidean').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)

10 loops, best of 5: 41 ms per loop


In [ ]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

100 loops, best of 5: 2.09 ms per loop


###  PCA + k-d Tree Algorithm  on one image

In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='kd_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)

10 loops, best of 5: 130 ms per loop


In [ ]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

100 loops, best of 5: 2.72 ms per loop


###  PCA + Ball Tree Algorithm  on one image


In [ ]:
%timeit NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)

10 loops, best of 5: 102 ms per loop


In [ ]:
%timeit neighbors.kneighbors([feature_list_compressed[random_image_index]])

1000 loops, best of 5: 1.87 ms per loop


Using the same random indices to experiment with 1000 set of images.

In [ ]:
random_feature_list_compressed = [
    feature_list_compressed[each_index] for each_index in random_image_indices
]

### PCA + Brute Force on 1000 images

In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='brute',
                             metric='euclidean').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1 loop, best of 5: 205 ms per loop


### PCA + k-d Tree Algorithm on 1000 images


In [ ]:
neighbors = NearestNeighbors(n_neighbors=5,
                             algorithm='kd_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1 loop, best of 5: 1.34 s per loop


### PCA + Ball Tree on 1000 images




In [ ]:
neighbors = NearestNeighbors(
    n_neighbors=5, algorithm='ball_tree').fit(feature_list_compressed)
%timeit neighbors.kneighbors(random_feature_list_compressed)

1 loop, best of 5: 1.1 s per loop


### Annoy
Annoy (Approximate Nearest Neighbors Oh Yeah) is a C++ library with Python bindings for searching nearest neighbors. Synonymous with speed, it was released by Spotify and is used in production to serve its music recommendations.

In [7]:
# installing annoy
!pip3 install annoy

     |████████████████████████████████| 655kB 12.5MB/s 
  Created wheel for annoy: filename=annoy-1.17.0-cp37-cp37m-linux_x86_64.whl size=386982 sha256=92acdda9e1d72ce1b08987796ab83e271f79049e78369e64add7fc3091c5ac2a
  Stored in directory: /root/.cache/pip/wheels/3a/c5/59/cce7e67b52c8e987389e53f917b6bb2a9d904a03246fadcb1e
Successfully built annoy


In [8]:
from annoy import AnnoyIndex

In [9]:
# Time the indexing for Annoy
t = AnnoyIndex(2048) # Length of feature vector that is to be indexed
starttime = time.time()
for i in range(num_images):
  feature = feature_list[i]
  t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(50) # 50 trees to build
t.save('/caltech101index.ann')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  


4.00521445274353


True

### Annoy on one image

In [10]:
u = AnnoyIndex(2048)
%timeit u.get_nns_by_vector(feature_list[random_image_index], 5, include_distances = True)
indexes = u.get_nns_by_vector(feature_list[random_image_index], 5,
                            include_distances = True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


1000 loops, best of 5: 460 µs per loop


Helper function to time the search for multiple images for Annoy. Perform the search for the same images multiple times to get an average value.

In [11]:
def calculate_annoy_time():
    for i in range(0, 1000):
        indexes = u.get_nns_by_vector(feature_list[random_image_index],
                                      5,
                                      include_distances=True)

### Annoy on 1000 images

In [12]:
%timeit calculate_annoy_time()

1 loop, best of 5: 454 ms per loop


### PCA + Annoy

In [17]:
starttime = time.time()
# Length of item vector that will be indexed
t = AnnoyIndex(num_feature_dimensions)

for i in range(num_images):
  feature = feature_list_compressed[i]
  t.add_item(i, feature)
endtime = time.time()
print(endtime - starttime)
t.build(50) # 50 trees
t.save('/caltech101index-pca.ann')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  This is separate from the ipykernel package so we can avoid doing imports until


0.0298306941986084


True

### PCA + Annoy for one image

In [19]:
u = AnnoyIndex(num_feature_dimensions)
%timeit u.get_nns_by_vector(feature_list_compressed[random_image_index], 5, include_distances=True)
indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index], 5,
                            include_distances=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  """Entry point for launching an IPython kernel.


The slowest run took 20.73 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 2.21 µs per loop


Again, writing a helper function to get an average time to perform search for the same image, 1000 times.

In [20]:
def calculate_annoy_time():
    for i in range(0, 1000):
        indexes = u.get_nns_by_vector(feature_list_compressed[random_image_index],
                                      5,
                                      include_distances=True)

### PCA + Annoy on 1000 images

In [21]:
%timeit calculate_annoy_time()

100 loops, best of 5: 2.42 ms per loop
